# Introduction

Welcome to this tutorial on using jupyter notebooks with Moveit 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial ().

In [1]:
import os
import sys
import yaml
import rclpy
import numpy as np

# message libraries
from geometry_msgs.msg import PoseStamped, Pose

# moveit_py
from moveit_py.planning import MoveItPy
from moveit_py.core import RobotState

# config file libraries
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory


# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
        MoveItConfigsBuilder(robot_name="panda", package_name="moveit_resources_panda_moveit_config")
        .robot_description(file_path="config/panda.urdf.xacro")
        .trajectory_execution(file_path="config/gripper_moveit_controllers.yaml")
        .moveit_cpp(
            file_path=get_package_share_directory("moveit2_tutorials")
            + "/config/jupyter_notebook_prototyping.yaml"
        )
        .to_moveit_configs()
    ).to_dict()

Failed to read parameter file from command line arguments.


## Setup

In [2]:
# initialise rclpy (only for logging purposes)
rclpy.init()

# instantiate moveit_py instance and a planning component for the panda_arm
moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
panda_arm = moveit.get_planning_component("panda_arm")

[INFO] [1672601742.400740052] [moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1672601742.400953453] [moveit_cpp_initializer]: Initialize node parameters
[INFO] [1672601742.464239974] [moveit_cpp_initializer]: params_filepath: /tmp/launch_params_3uz30lly
[INFO] [1672601742.464317689] [moveit_cpp_initializer]: Initialize node and executor
[INFO] [1672601742.519777951] [moveit_cpp_initializer]: Spin separate thread
[INFO] [1672601742.522371797] [moveit_rdf_loader.rdf_loader]: Loaded robot model in 0.0023126 seconds
[INFO] [1672601742.522497118] [moveit_robot_model.robot_model]: Loading robot model 'panda'...
[INFO] [1672601742.557053458] [kdl_parser]: Link panda_link1 had 1 children.
[INFO] [1672601742.557100115] [kdl_parser]: Link panda_link2 had 1 children.
[INFO] [1672601742.557107585] [kdl_parser]: Link panda_link3 had 1 children.
[INFO] [1672601742.557113979] [kdl_parser]: Link panda_link4 had 1 children.
[INFO] [1672601742.557121696] [kdl_parser]: Link panda_link5 had 1 children

## Motion Planning Example

In [3]:
# set plan start state using predefined state
panda_arm.set_start_state("ready")

# set pose goal using predefined state
panda_arm.set_goal(goal_state_name = "extended")

# plan to goal
plan_result = panda_arm.plan()

# execute the plan
if plan_result:
    panda_arm.execute()

[INFO] [1672601743.925636638] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'panda_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1672601748.667177958] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601748.667305907] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601748.667658250] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601748.667818852] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601748.668080547] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1672601748.674987710] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1672601748.675067910] [moveit.plugins.moveit_simple_controller_manag

Great, so we can perform motion planning interactively (see the motion planning tutorial for further details of the motion planning API).Suppose we are developing our code and we make a mistake such as follows:

In [4]:
# set plan start state using predefined state
panda_arm.set_start_state("ready") # This conflicts with the current robot configuration and will cause and error

# set goal using a pose message this time
pose_goal = PoseStamped()
pose_goal.header.frame_id = "panda_link0"
pose_goal.pose.orientation.w = 1.0
pose_goal.pose.position.x = 0.28
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = 0.5
panda_arm.set_goal(goal_pose_msg = pose_goal, link_name = "panda_link8")

# plan to goal
plan_result = panda_arm.plan()

# execute the plan
if plan_result:
    panda_arm.execute()

[INFO] [1672601750.590561378] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'panda_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1672601756.186112750] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601756.186261804] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601756.186632296] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601756.186751599] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601756.187077996] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[ERROR] [1672601756.195713962] [moveit_ros.trajectory_execution_manager]: 
Invalid Trajectory: start point deviates from current robot state more than 0.01
joint 'panda_joint2': expec

Since we are using a notebook this mistake is easy to rectify without having to fix the bug and recompile files. Simply edit the above notebook to match the below and rerun the cell.

In [5]:
# set plan start state using predefined state
panda_arm.set_start_state_to_current_state() # This conflicts with the current robot pose and will cause and error

# set goal using a pose message this time
pose_goal = PoseStamped()
pose_goal.header.frame_id = "panda_link0"
pose_goal.pose.orientation.w = 1.0
pose_goal.pose.position.x = 0.28
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = 0.5
panda_arm.set_goal(goal_pose_msg = pose_goal, link_name = "panda_link8")

# plan to goal
plan_result = panda_arm.plan()

# execute the plan
if plan_result:
    panda_arm.execute()

[INFO] [1672601756.218550249] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'panda_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1672601761.745787127] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601761.745881222] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601761.746099186] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601761.746133666] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1672601761.746325917] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1672601761.754883916] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1672601761.754955646] [moveit.plugins.moveit_simple_controller_manag

## Teleoperation Example

One may also want to perform live teleoperation of their robot. Wouldn't it be nice if we could interactively start/stop teleoperation without shutting down and subsequently relaunching all processes. In this example, we are going to show how this is possible with notebooks through a motivating example of teleoperating the robot, performing motion planning and teleoperating the robot again. 

For this section you will need a device that support teleoperation with moveit_py, in this case we leverage the PS4 dualshock controller. 

In [ ]:
# launch servo client from within notebook
!ros2 launch moveit2_tutorials notebook_start_servo_client.launch.py

In [ ]:
from moveit_py.servo_client.devices.ps4_dualshock import PS4DualShockTeleop

# instantiate the teleoperating device
ps4 = PS4DualShockTeleop()

# start teleloperating the robot
ps4.start_teleop()

Oh wait but I actually want to try perform some motion planning to bring the robot back to its default configuration, no problem...

In [ ]:
# stop teleoperating the robot
ps4.stop_teleop()

# plan and execute
# set plan start state using predefined state
panda_arm.set_start_state_to_current_state()

# set pose goal using predefined state
panda_arm.set_goal(goal_state_name = "ready")

# plan to goal
plan_result = panda_arm.plan()

# execute the plan
if plan_result:
    panda_arm.execute()

Ok great now we are back at our default configuration, lets start teleoperating the robot again.

In [ ]:
ps4.start_teleop()